In [1]:
import pickle
import os
import re
import numpy as np
from gensim.models import word2vec
import logging
import pandas as pd

In [2]:
import sys
sys.path.insert(0, '../Core-scripts/')

from parse_and_prepare import ProteinProteinInteractionClassifier as ppi
import file_readers as fr
import prediction as pred

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

In [4]:
random_seeds = [144, 235, 905, 2895, 3462, 4225, 5056, 5192, 7751, 7813]

In [5]:
def make_w2v_model(dataset, name_for_model, model_features=None):
    """Produce a Word2Vec Model

    Model_features (list): Features of the word to vec models
        1. Word vector dimensionality
        2. Minimum word count
        3. Number of threads to run in parallel
        4. Context window size
        5. Downsample setting for frequent words

    """

    print ('Parsing datasets sentences')

    sentences = [fr.sentence_to_wordlist(sen) for sen in dataset]

    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

    # Set values for various parameters
    if model_features:
        num_features = model_features[0] #300  # Word vector dimensionality
        min_word_count = model_features[1] #5  # Minimum word count
        num_workers = model_features[2] #4  # Number of threads to run in parallel
        context = model_features[3] #6  # Context window size
        downsampling = model_features[4] #0.001  # Downsample setting for frequent words
    else:
        num_features = 600  # Word vector dimensionality
        min_word_count = 6  # Minimum word count
        num_workers = 4  # Number of threads to run in parallel
        context = 7  # Context window size
        downsampling = 0.0001  # Downsample setting for frequent words

    print('Training Word2Vec Model')

    model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count=min_word_count, \
            window=context, sample=downsampling)

    # If you don't plan to train the model any further, calling
    # init_sims will make the model much more memory-efficient.
    model.init_sims(replace=False)

    model_name = 'Results/' + name_for_model + '_model'

    model.save(model_name)

    w2v_model = model

    return w2v_model

In [5]:
# Yeast using old model (w/o skipgramms and hierarchical softmax)

yeast_strict = pickle.load(open('../../Results/Yeast/yeast_mentions_strict_real.pkl', 'rb'))
yeast_gen = pickle.load(open('../../Results/Yeast/yeast_mentions_gen_real.pkl', 'rb'))
yeast_be = pickle.load(open('../../Results/Yeast/yeast_mentions_be_real.pkl', 'rb'))

In [6]:
yeast_strict_model = pred.make_w2v_model(yeast_strict, 'yeast_strict_old')
yeast_gen_model = pred.make_w2v_model(yeast_gen, 'yeast_gen_old')
yeast_be_model = pred.make_w2v_model(yeast_be, 'yeast_be_old')

2017-06-06 19:01:05,387 : INFO : collecting all words and their counts
2017-06-06 19:01:05,387 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-06 19:01:05,419 : INFO : collected 11431 word types from a corpus of 188505 raw words and 7145 sentences
2017-06-06 19:01:05,420 : INFO : Loading a fresh vocabulary
2017-06-06 19:01:05,429 : INFO : min_count=7 retains 2632 unique words (23% of original 11431, drops 8799)
2017-06-06 19:01:05,430 : INFO : min_count=7 leaves 171509 word corpus (90% of original 188505, drops 16996)
2017-06-06 19:01:05,436 : INFO : deleting the raw counts dictionary of 11431 items
2017-06-06 19:01:05,438 : INFO : sample=0.001 downsamples 42 most-common words
2017-06-06 19:01:05,438 : INFO : downsampling leaves estimated 121427 word corpus (70.8% of prior 171509)
2017-06-06 19:01:05,439 : INFO : estimated required memory for 2632 words and 800 dimensions: 18160800 bytes
2017-06-06 19:01:05,445 : INFO : resetting layer weights
2017-06

Parsing datasets sentences
Training Word2Vec Model


2017-06-06 19:01:06,507 : INFO : PROGRESS: at 83.71% examples, 506480 words/s, in_qsize 15, out_qsize 0
2017-06-06 19:01:06,630 : INFO : worker thread finished; awaiting finish of 7 more threads
2017-06-06 19:01:06,635 : INFO : worker thread finished; awaiting finish of 6 more threads
2017-06-06 19:01:06,654 : INFO : worker thread finished; awaiting finish of 5 more threads
2017-06-06 19:01:06,659 : INFO : worker thread finished; awaiting finish of 4 more threads
2017-06-06 19:01:06,662 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-06-06 19:01:06,666 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-06-06 19:01:06,675 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-06-06 19:01:06,683 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-06-06 19:01:06,684 : INFO : training on 942525 raw words (607395 effective words) took 1.2s, 514538 effective words/s
2017-06-06 19:01:06,685 : INFO : precomputing

Parsing datasets sentences


2017-06-06 19:01:07,554 : INFO : collecting all words and their counts
2017-06-06 19:01:07,555 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-06 19:01:07,628 : INFO : PROGRESS: at sentence #10000, processed 242823 words, keeping 11175 word types
2017-06-06 19:01:07,697 : INFO : PROGRESS: at sentence #20000, processed 485524 words, keeping 14974 word types


...

2017-06-06 19:01:07,767 : INFO : PROGRESS: at sentence #30000, processed 726123 words, keeping 17544 word types
2017-06-06 19:01:07,829 : INFO : PROGRESS: at sentence #40000, processed 970457 words, keeping 19716 word types
2017-06-06 19:01:07,864 : INFO : collected 21147 word types from a corpus of 1155938 raw words and 47662 sentences
2017-06-06 19:01:07,864 : INFO : Loading a fresh vocabulary
2017-06-06 19:01:07,891 : INFO : min_count=7 retains 6774 unique words (32% of original 21147, drops 14373)
2017-06-06 19:01:07,891 : INFO : min_count=7 leaves 1127019 word corpus (97% of original 1155938, drops 28919)
2017-06-06 19:01:07,912 : INFO : deleting the raw counts dictionary of 21147 items
2017-06-06 19:01:07,914 : INFO : sample=0.001 downsamples 48 most-common words
2017-06-06 19:01:07,914 : INFO : downsampling leaves estimated 821525 word corpus (72.9% of prior 1127019)
2017-06-06 19:01:07,915 : INFO : estimated required memory for 6774 words and 800 dimensions: 46740600 bytes
2017

Parsing datasets sentences


2017-06-06 19:01:22,220 : INFO : collecting all words and their counts
2017-06-06 19:01:22,221 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-06 19:01:22,267 : INFO : PROGRESS: at sentence #10000, processed 229556 words, keeping 12380 word types
2017-06-06 19:01:22,314 : INFO : PROGRESS: at sentence #20000, processed 458882 words, keeping 16959 word types
2017-06-06 19:01:22,361 : INFO : PROGRESS: at sentence #30000, processed 688033 words, keeping 20220 word types
2017-06-06 19:01:22,410 : INFO : PROGRESS: at sentence #40000, processed 915106 words, keeping 22835 word types


Training Word2Vec Model


2017-06-06 19:01:22,458 : INFO : PROGRESS: at sentence #50000, processed 1143809 words, keeping 25149 word types
2017-06-06 19:01:22,508 : INFO : PROGRESS: at sentence #60000, processed 1373267 words, keeping 27064 word types
2017-06-06 19:01:22,559 : INFO : PROGRESS: at sentence #70000, processed 1602253 words, keeping 28904 word types
2017-06-06 19:01:22,617 : INFO : PROGRESS: at sentence #80000, processed 1831769 words, keeping 30605 word types
2017-06-06 19:01:22,682 : INFO : PROGRESS: at sentence #90000, processed 2061068 words, keeping 32225 word types
2017-06-06 19:01:22,746 : INFO : PROGRESS: at sentence #100000, processed 2289682 words, keeping 33663 word types
2017-06-06 19:01:22,809 : INFO : PROGRESS: at sentence #110000, processed 2519258 words, keeping 34997 word types
2017-06-06 19:01:22,861 : INFO : PROGRESS: at sentence #120000, processed 2746740 words, keeping 36260 word types
2017-06-06 19:01:22,911 : INFO : PROGRESS: at sentence #130000, processed 2976233 words, keep

In [8]:
import time
name_of_result = 'yeast_OLD'
strict_data = yeast_strict
w2v_strict = yeast_strict_model
w2v_gen = yeast_gen_model
w2v_be = yeast_be_model
xgb_clf = XGBClassifier(seed=24)
start = time.time()
for seed in random_seeds:
        strict_list_SR = pred.make_models(strict_data,
                                          name_of_result+'_SR_'+str(seed),
                                          prev_model=w2v_strict,
                                          ran_state=seed)

        strict_list_GEN = pred.make_models(strict_data,
                                           name_of_result+'_GEN_'+str(seed),
                                           prev_model=w2v_gen,
                                           ran_state=seed)
        strict_list_BE = pred.make_models(strict_data,
                                          name_of_result+'_BE_'+str(seed),
                                          prev_model=w2v_be,
                                          ran_state=seed)

        strict_final_list = [strict_list_SR,
                             strict_list_GEN,
                             strict_list_BE]

        print ('\nPredicting\n')
        accuracy = []
        probs = []
        fpr = []
        tpr = []
        labels = []
        auc_score = []
        report = []

        for entry, model_name in zip(strict_final_list, ['SR '+str(seed), 'GEN '+str(seed), 'BE '+str(seed)]):
            accuracy_norm, auc_score_norm, pred_labels_norm, probs_norm, class_report_norm  = pred.XGB_modelfit(xgb_clf, 
                                                                                                                entry[0], 
                                                                                                                entry[2], 
                                                                                                                entry[1], 
                                                                                                                entry[3], 
                                                                                                                model_name)
            fpr_norm, tpr_norm, _ = roc_curve(entry[3], probs_norm)

            accuracy.append([accuracy_norm])
            probs.append([probs_norm])
            fpr.append([fpr_norm])
            tpr.append([tpr_norm])
            labels.append([pred_labels_norm])
            auc_score.append([auc_score_norm])
            report.append([class_report_norm])

        pickle.dump(accuracy, open('Results/'+name_of_result+'_accuracy_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        pickle.dump(probs, open('Results/'+name_of_result+'_probs_pickle_'+str(seed)+'.pkl',
                                'wb'))
        pickle.dump(fpr, open('Results/'+name_of_result+'_fpr_pickle_'+str(seed)+'.pkl',
                              'wb'))
        pickle.dump(tpr, open('Results/'+name_of_result+'_tpr_pickle_'+str(seed)+'.pkl',
                              'wb'))
        pickle.dump(labels, open('Results/'+name_of_result+'_labels_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        pickle.dump(auc_score, open('Results/'+name_of_result+'_auc_score_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        pickle.dump(report, open('Results/'+name_of_result+'_report_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        
print('Took ', time.time()-start, ' seconds')

../Core-scripts/prediction.py:178: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 144 
Model Report
SR 144 Accuracy: 0.7177
SR 144 AUC Score (Train): 0.737143
SR 144 Report 
              precision    recall  f1-score   support

          0       0.74      0.83      0.78        75
          1       0.68      0.55      0.61        49

avg / total       0.71      0.72      0.71       124

GEN 144 
Model Report
GEN 144 Accuracy: 0.7984
GEN 144 AUC Score (Train): 0.842721
GEN 144 Report 
              precision    recall  f1-score   support

          0       0.78      0.92      0.85        75
          1       0.83      0.61      0.71        49

avg / total       0.80      0.80      0.79       124

BE 144 
Model Report
BE 144 Accuracy: 0.7661
BE 144 AUC Score (Train): 0.857415
BE 144 Report 
              precision    recall  f1-score   support

          0       0.77      0.88      0.82        75
          1       0.76      0.59      0.67        49

avg / total       0.77      0.77      0.76       124



../Core-scripts/prediction.py:178: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 235 
Model Report
SR 235 Accuracy: 0.6667
SR 235 AUC Score (Train): 0.762759
SR 235 Report 
              precision    recall  f1-score   support

          0       0.64      0.86      0.74        29
          1       0.73      0.44      0.55        25

avg / total       0.68      0.67      0.65        54

GEN 235 
Model Report
GEN 235 Accuracy: 0.7037
GEN 235 AUC Score (Train): 0.866207
GEN 235 Report 
              precision    recall  f1-score   support

          0       0.67      0.90      0.76        29
          1       0.80      0.48      0.60        25

avg / total       0.73      0.70      0.69        54

BE 235 
Model Report
BE 235 Accuracy: 0.7407
BE 235 AUC Score (Train): 0.877241
BE 235 Report 
              precision    recall  f1-score   support

          0       0.70      0.90      0.79        29
          1       0.82      0.56      0.67        25

avg / total       0.76      0.74      0.73        54



../Core-scripts/prediction.py:178: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)


...


Predicting

SR 5056 
Model Report
SR 5056 Accuracy: 0.6232
SR 5056 AUC Score (Train): 0.633157
SR 5056 Report 
              precision    recall  f1-score   support

          0       0.67      0.74      0.70        42
          1       0.52      0.44      0.48        27

avg / total       0.61      0.62      0.62        69

GEN 5056 
Model Report
GEN 5056 Accuracy: 0.6667
GEN 5056 AUC Score (Train): 0.694885
GEN 5056 Report 
              precision    recall  f1-score   support

          0       0.71      0.76      0.74        42
          1       0.58      0.52      0.55        27

avg / total       0.66      0.67      0.66        69

BE 5056 
Model Report
BE 5056 Accuracy: 0.6957
BE 5056 AUC Score (Train): 0.742504
BE 5056 Report 
              precision    recall  f1-score   support

          0       0.73      0.79      0.76        42
          1       0.62      0.56      0.59        27

avg / total       0.69      0.70      0.69        69



../Core-scripts/prediction.py:178: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 5192 
Model Report
SR 5192 Accuracy: 0.4848
SR 5192 AUC Score (Train): 0.518269
SR 5192 Report 
              precision    recall  f1-score   support

          0       0.40      0.62      0.48        26
          1       0.62      0.40      0.48        40

avg / total       0.53      0.48      0.48        66

GEN 5192 
Model Report
GEN 5192 Accuracy: 0.6212
GEN 5192 AUC Score (Train): 0.628846
GEN 5192 Report 
              precision    recall  f1-score   support

          0       0.52      0.58      0.55        26
          1       0.70      0.65      0.68        40

avg / total       0.63      0.62      0.62        66

BE 5192 
Model Report
BE 5192 Accuracy: 0.6061
BE 5192 AUC Score (Train): 0.601923
BE 5192 Report 
              precision    recall  f1-score   support

          0       0.50      0.62      0.55        26
          1       0.71      0.60      0.65        40

avg / total       0.62      0.61      0.61        66



../Core-scripts/prediction.py:178: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 7751 
Model Report
SR 7751 Accuracy: 0.6905
SR 7751 AUC Score (Train): 0.748843
SR 7751 Report 
              precision    recall  f1-score   support

          0       0.70      0.81      0.75        48
          1       0.68      0.53      0.59        36

avg / total       0.69      0.69      0.68        84

GEN 7751 
Model Report
GEN 7751 Accuracy: 0.7619
GEN 7751 AUC Score (Train): 0.757523
GEN 7751 Report 
              precision    recall  f1-score   support

          0       0.74      0.90      0.81        48
          1       0.81      0.58      0.68        36

avg / total       0.77      0.76      0.75        84

BE 7751 
Model Report
BE 7751 Accuracy: 0.7262
BE 7751 AUC Score (Train): 0.815972
BE 7751 Report 
              precision    recall  f1-score   support

          0       0.72      0.85      0.78        48
          1       0.74      0.56      0.63        36

avg / total       0.73      0.73      0.72        84


Predicting

SR 7813 
Model Report
SR 

In [7]:
import time
name_of_result = 'yeast_OLD'
strict_data = yeast_strict
w2v_strict = yeast_strict_model
w2v_gen = yeast_gen_model
w2v_be = yeast_be_model
xgb_clf = XGBClassifier(seed=24)
start = time.time()
for seed in random_seeds:
        strict_list_SR = pred.make_models(strict_data,
                                          name_of_result+'_SR_'+str(seed),
                                          prev_model=w2v_strict,
                                          ran_state=seed)

        strict_list_GEN = pred.make_models(strict_data,
                                           name_of_result+'_GEN_'+str(seed),
                                           prev_model=w2v_gen,
                                           ran_state=seed)
        strict_list_BE = pred.make_models(strict_data,
                                          name_of_result+'_BE_'+str(seed),
                                          prev_model=w2v_be,
                                          ran_state=seed)

        strict_final_list = [strict_list_SR,
                             strict_list_GEN,
                             strict_list_BE]

        print ('\nPredicting\n')
        accuracy = []
        probs = []
        fpr = []
        tpr = []
        labels = []
        auc_score = []
        report = []

        for entry, model_name in zip(strict_final_list, ['SR '+str(seed), 'GEN '+str(seed), 'BE '+str(seed)]):
            accuracy_norm, auc_score_norm, pred_labels_norm, probs_norm, class_report_norm  = pred.XGB_modelfit(xgb_clf, 
                                                                                                                entry[0], 
                                                                                                                entry[2], 
                                                                                                                entry[1], 
                                                                                                                entry[3], 
                                                                                                                model_name)
            fpr_norm, tpr_norm, _ = roc_curve(entry[3], probs_norm)

            accuracy.append([accuracy_norm])
            probs.append([probs_norm])
            fpr.append([fpr_norm])
            tpr.append([tpr_norm])
            labels.append([pred_labels_norm])
            auc_score.append([auc_score_norm])
            report.append([class_report_norm])

        pickle.dump(accuracy, open('Results/'+name_of_result+'_accuracy_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        pickle.dump(probs, open('Results/'+name_of_result+'_probs_pickle_'+str(seed)+'.pkl',
                                'wb'))
        pickle.dump(fpr, open('Results/'+name_of_result+'_fpr_pickle_'+str(seed)+'.pkl',
                              'wb'))
        pickle.dump(tpr, open('Results/'+name_of_result+'_tpr_pickle_'+str(seed)+'.pkl',
                              'wb'))
        pickle.dump(labels, open('Results/'+name_of_result+'_labels_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        pickle.dump(auc_score, open('Results/'+name_of_result+'_auc_score_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        pickle.dump(report, open('Results/'+name_of_result+'_report_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        
print('Took ', time.time()-start, ' seconds')


Predicting

SR 144 
Model Report
SR 144 Accuracy: 0.5581
SR 144 AUC Score (Train): 0.578049
SR 144 Report 
              precision    recall  f1-score   support

          0       0.56      0.71      0.63        45
          1       0.55      0.39      0.46        41

avg / total       0.56      0.56      0.55        86

GEN 144 
Model Report
GEN 144 Accuracy: 0.5233
GEN 144 AUC Score (Train): 0.583198
GEN 144 Report 
              precision    recall  f1-score   support

          0       0.54      0.67      0.59        45
          1       0.50      0.37      0.42        41

avg / total       0.52      0.52      0.51        86

BE 144 
Model Report
BE 144 Accuracy: 0.5465
BE 144 AUC Score (Train): 0.583740
BE 144 Report 
              precision    recall  f1-score   support

          0       0.55      0.73      0.63        45
          1       0.54      0.34      0.42        41

avg / total       0.54      0.55      0.53        86



../Core-scripts/prediction.py:178: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 235 
Model Report
SR 235 Accuracy: 0.6
SR 235 AUC Score (Train): 0.666980
SR 235 Report 
              precision    recall  f1-score   support

          0       0.59      0.76      0.67        42
          1       0.62      0.42      0.50        38

avg / total       0.60      0.60      0.59        80

GEN 235 
Model Report
GEN 235 Accuracy: 0.6125
GEN 235 AUC Score (Train): 0.633459
GEN 235 Report 
              precision    recall  f1-score   support

          0       0.60      0.76      0.67        42
          1       0.63      0.45      0.52        38

avg / total       0.62      0.61      0.60        80

BE 235 
Model Report
BE 235 Accuracy: 0.6
BE 235 AUC Score (Train): 0.646617
BE 235 Report 
              precision    recall  f1-score   support

          0       0.60      0.74      0.66        42
          1       0.61      0.45      0.52        38

avg / total       0.60      0.60      0.59        80



../Core-scripts/prediction.py:178: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 905 
Model Report
SR 905 Accuracy: 0.5974
SR 905 AUC Score (Train): 0.736395
SR 905 Report 
              precision    recall  f1-score   support

          0       0.58      0.93      0.72        42
          1       0.70      0.20      0.31        35

avg / total       0.64      0.60      0.53        77

GEN 905 
Model Report
GEN 905 Accuracy: 0.6104
GEN 905 AUC Score (Train): 0.751701
GEN 905 Report 
              precision    recall  f1-score   support

          0       0.60      0.83      0.70        42
          1       0.63      0.34      0.44        35

avg / total       0.62      0.61      0.58        77

BE 905 
Model Report
BE 905 Accuracy: 0.6234
BE 905 AUC Score (Train): 0.763946
BE 905 Report 
              precision    recall  f1-score   support

          0       0.61      0.83      0.71        42
          1       0.65      0.37      0.47        35

avg / total       0.63      0.62      0.60        77


Predicting

SR 2895 
Model Report
SR 2895 Accurac

...


Predicting

SR 5192 
Model Report
SR 5192 Accuracy: 0.5455
SR 5192 AUC Score (Train): 0.611111
SR 5192 Report 
              precision    recall  f1-score   support

          0       0.41      0.58      0.48        24
          1       0.69      0.52      0.59        42

avg / total       0.59      0.55      0.55        66

GEN 5192 
Model Report
GEN 5192 Accuracy: 0.5758
GEN 5192 AUC Score (Train): 0.631944
GEN 5192 Report 
              precision    recall  f1-score   support

          0       0.44      0.67      0.53        24
          1       0.73      0.52      0.61        42

avg / total       0.63      0.58      0.58        66

BE 5192 
Model Report
BE 5192 Accuracy: 0.6818
BE 5192 AUC Score (Train): 0.716270
BE 5192 Report 
              precision    recall  f1-score   support

          0       0.54      0.83      0.66        24
          1       0.86      0.60      0.70        42

avg / total       0.75      0.68      0.69        66



../Core-scripts/prediction.py:178: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 7751 
Model Report
SR 7751 Accuracy: 0.7727
SR 7751 AUC Score (Train): 0.750000
SR 7751 Report 
              precision    recall  f1-score   support

          0       0.81      0.90      0.85        48
          1       0.62      0.44      0.52        18

avg / total       0.76      0.77      0.76        66

GEN 7751 
Model Report
GEN 7751 Accuracy: 0.7576
GEN 7751 AUC Score (Train): 0.822917
GEN 7751 Report 
              precision    recall  f1-score   support

          0       0.86      0.79      0.83        48
          1       0.55      0.67      0.60        18

avg / total       0.78      0.76      0.76        66

BE 7751 
Model Report
BE 7751 Accuracy: 0.8333
BE 7751 AUC Score (Train): 0.866898
BE 7751 Report 
              precision    recall  f1-score   support

          0       0.88      0.90      0.89        48
          1       0.71      0.67      0.69        18

avg / total       0.83      0.83      0.83        66



../Core-scripts/prediction.py:178: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 7813 
Model Report
SR 7813 Accuracy: 0.5893
SR 7813 AUC Score (Train): 0.678161
SR 7813 Report 
              precision    recall  f1-score   support

          0       0.56      0.67      0.61        27
          1       0.62      0.52      0.57        29

avg / total       0.59      0.59      0.59        56

GEN 7813 
Model Report
GEN 7813 Accuracy: 0.5893
GEN 7813 AUC Score (Train): 0.664112
GEN 7813 Report 
              precision    recall  f1-score   support

          0       0.56      0.67      0.61        27
          1       0.62      0.52      0.57        29

avg / total       0.59      0.59      0.59        56

BE 7813 
Model Report
BE 7813 Accuracy: 0.5357
BE 7813 AUC Score (Train): 0.609195
BE 7813 Report 
              precision    recall  f1-score   support

          0       0.51      0.67      0.58        27
          1       0.57      0.41      0.48        29

avg / total       0.54      0.54      0.53        56

Took  301.4283707141876  seconds


In [7]:
import time
name_of_result = 'yeast_OLD'
strict_data = yeast_strict
w2v_strict = yeast_strict_model
w2v_gen = yeast_gen_model
w2v_be = yeast_be_model
xgb_clf = XGBClassifier(seed=24)
start = time.time()
for seed in random_seeds:
        strict_list_SR = pred.make_models(strict_data,
                                          name_of_result+'_SR_'+str(seed),
                                          prev_model=w2v_strict,
                                          ran_state=seed)

        strict_list_GEN = pred.make_models(strict_data,
                                           name_of_result+'_GEN_'+str(seed),
                                           prev_model=w2v_gen,
                                           ran_state=seed)
        strict_list_BE = pred.make_models(strict_data,
                                          name_of_result+'_BE_'+str(seed),
                                          prev_model=w2v_be,
                                          ran_state=seed)

        strict_final_list = [strict_list_SR,
                             strict_list_GEN,
                             strict_list_BE]

        print ('\nPredicting\n')
        accuracy = []
        probs = []
        fpr = []
        tpr = []
        labels = []
        auc_score = []
        report = []

        for entry, model_name in zip(strict_final_list, ['SR '+str(seed), 'GEN '+str(seed), 'BE '+str(seed)]):
            accuracy_norm, auc_score_norm, pred_labels_norm, probs_norm, class_report_norm  = pred.XGB_modelfit(xgb_clf, 
                                                                                                                entry[0], 
                                                                                                                entry[2], 
                                                                                                                entry[1], 
                                                                                                                entry[3], 
                                                                                                                model_name)
            fpr_norm, tpr_norm, _ = roc_curve(entry[3], probs_norm)

            accuracy.append([accuracy_norm])
            probs.append([probs_norm])
            fpr.append([fpr_norm])
            tpr.append([tpr_norm])
            labels.append([pred_labels_norm])
            auc_score.append([auc_score_norm])
            report.append([class_report_norm])

        pickle.dump(accuracy, open('Results/'+name_of_result+'_accuracy_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        pickle.dump(probs, open('Results/'+name_of_result+'_probs_pickle_'+str(seed)+'.pkl',
                                'wb'))
        pickle.dump(fpr, open('Results/'+name_of_result+'_fpr_pickle_'+str(seed)+'.pkl',
                              'wb'))
        pickle.dump(tpr, open('Results/'+name_of_result+'_tpr_pickle_'+str(seed)+'.pkl',
                              'wb'))
        pickle.dump(labels, open('Results/'+name_of_result+'_labels_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        pickle.dump(auc_score, open('Results/'+name_of_result+'_auc_score_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        pickle.dump(report, open('Results/'+name_of_result+'_report_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        
print('Took ', time.time()-start, ' seconds')

../Core-scripts/prediction.py:179: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 144 
Model Report
SR 144 Accuracy: 0.6197
SR 144 AUC Score (Train): 0.730640
SR 144 Report 
              precision    recall  f1-score   support

          0       0.50      0.70      0.58        27
          1       0.76      0.57      0.65        44

avg / total       0.66      0.62      0.62        71

GEN 144 
Model Report
GEN 144 Accuracy: 0.6338
GEN 144 AUC Score (Train): 0.730640
GEN 144 Report 
              precision    recall  f1-score   support

          0       0.51      0.67      0.58        27
          1       0.75      0.61      0.67        44

avg / total       0.66      0.63      0.64        71

BE 144 
Model Report
BE 144 Accuracy: 0.7183
BE 144 AUC Score (Train): 0.734848
BE 144 Report 
              precision    recall  f1-score   support

          0       0.61      0.74      0.67        27
          1       0.82      0.70      0.76        44

avg / total       0.74      0.72      0.72        71



../Core-scripts/prediction.py:179: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 235 
Model Report
SR 235 Accuracy: 0.7167
SR 235 AUC Score (Train): 0.746250
SR 235 Report 
              precision    recall  f1-score   support

          0       0.83      0.72      0.77        40
          1       0.56      0.70      0.62        20

avg / total       0.74      0.72      0.72        60

GEN 235 
Model Report
GEN 235 Accuracy: 0.7833
GEN 235 AUC Score (Train): 0.806250
GEN 235 Report 
              precision    recall  f1-score   support

          0       0.85      0.82      0.84        40
          1       0.67      0.70      0.68        20

avg / total       0.79      0.78      0.78        60

BE 235 
Model Report
BE 235 Accuracy: 0.7667
BE 235 AUC Score (Train): 0.821250
BE 235 Report 
              precision    recall  f1-score   support

          0       0.81      0.85      0.83        40
          1       0.67      0.60      0.63        20

avg / total       0.76      0.77      0.76        60



../Core-scripts/prediction.py:179: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)


...


Predicting

SR 5192 
Model Report
SR 5192 Accuracy: 0.6905
SR 5192 AUC Score (Train): 0.732265
SR 5192 Report 
              precision    recall  f1-score   support

          0       0.69      0.80      0.74        46
          1       0.70      0.55      0.62        38

avg / total       0.69      0.69      0.68        84

GEN 5192 
Model Report
GEN 5192 Accuracy: 0.6667
GEN 5192 AUC Score (Train): 0.735698
GEN 5192 Report 
              precision    recall  f1-score   support

          0       0.70      0.70      0.70        46
          1       0.63      0.63      0.63        38

avg / total       0.67      0.67      0.67        84

BE 5192 
Model Report
BE 5192 Accuracy: 0.7143
BE 5192 AUC Score (Train): 0.775172
BE 5192 Report 
              precision    recall  f1-score   support

          0       0.72      0.78      0.75        46
          1       0.71      0.63      0.67        38

avg / total       0.71      0.71      0.71        84



../Core-scripts/prediction.py:179: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 7751 
Model Report
SR 7751 Accuracy: 0.5833
SR 7751 AUC Score (Train): 0.608081
SR 7751 Report 
              precision    recall  f1-score   support

          0       0.70      0.70      0.70        33
          1       0.33      0.33      0.33        15

avg / total       0.58      0.58      0.58        48

GEN 7751 
Model Report
GEN 7751 Accuracy: 0.7083
GEN 7751 AUC Score (Train): 0.828283
GEN 7751 Report 
              precision    recall  f1-score   support

          0       0.79      0.79      0.79        33
          1       0.53      0.53      0.53        15

avg / total       0.71      0.71      0.71        48

BE 7751 
Model Report
BE 7751 Accuracy: 0.6875
BE 7751 AUC Score (Train): 0.785859
BE 7751 Report 
              precision    recall  f1-score   support

          0       0.78      0.76      0.77        33
          1       0.50      0.53      0.52        15

avg / total       0.69      0.69      0.69        48



../Core-scripts/prediction.py:179: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 7813 
Model Report
SR 7813 Accuracy: 0.6731
SR 7813 AUC Score (Train): 0.692187
SR 7813 Report 
              precision    recall  f1-score   support

          0       0.71      0.78      0.75        32
          1       0.59      0.50      0.54        20

avg / total       0.67      0.67      0.67        52

GEN 7813 
Model Report
GEN 7813 Accuracy: 0.5577
GEN 7813 AUC Score (Train): 0.629687
GEN 7813 Report 
              precision    recall  f1-score   support

          0       0.65      0.62      0.63        32
          1       0.43      0.45      0.44        20

avg / total       0.56      0.56      0.56        52

BE 7813 
Model Report
BE 7813 Accuracy: 0.6154
BE 7813 AUC Score (Train): 0.645312
BE 7813 Report 
              precision    recall  f1-score   support

          0       0.71      0.62      0.67        32
          1       0.50      0.60      0.55        20

avg / total       0.63      0.62      0.62        52

Took  762.2354707717896  seconds


In [9]:
import time
name_of_result = 'yeast_OLD'
strict_data = yeast_strict
w2v_strict = yeast_strict_model
w2v_gen = yeast_gen_model
w2v_be = yeast_be_model
xgb_clf = XGBClassifier(learning_rate = 0.1,
                        n_estimators = 1000,
                        seed=24)
start = time.time()
for seed in random_seeds:
        strict_list_SR = pred.make_models(strict_data,
                                          name_of_result+'_SR_'+str(seed),
                                          prev_model=w2v_strict,
                                          ran_state=seed)

        strict_list_GEN = pred.make_models(strict_data,
                                           name_of_result+'_GEN_'+str(seed),
                                           prev_model=w2v_gen,
                                           ran_state=seed)
        strict_list_BE = pred.make_models(strict_data,
                                          name_of_result+'_BE_'+str(seed),
                                          prev_model=w2v_be,
                                          ran_state=seed)

        strict_final_list = [strict_list_SR,
                             strict_list_GEN,
                             strict_list_BE]

        print ('\nPredicting\n')
        accuracy = []
        probs = []
        fpr = []
        tpr = []
        labels = []
        auc_score = []
        report = []

        for entry, model_name in zip(strict_final_list, ['SR '+str(seed), 'GEN '+str(seed), 'BE '+str(seed)]):
            accuracy_norm, auc_score_norm, pred_labels_norm, probs_norm, class_report_norm  = pred.XGB_modelfit(xgb_clf, 
                                                                                                                entry[0], 
                                                                                                                entry[2], 
                                                                                                                entry[1], 
                                                                                                                entry[3], 
                                                                                                                model_name)
            fpr_norm, tpr_norm, _ = roc_curve(entry[3], probs_norm)

            accuracy.append([accuracy_norm])
            probs.append([probs_norm])
            fpr.append([fpr_norm])
            tpr.append([tpr_norm])
            labels.append([pred_labels_norm])
            auc_score.append([auc_score_norm])
            report.append([class_report_norm])

        pickle.dump(accuracy, open('Results/'+name_of_result+'_accuracy_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        pickle.dump(probs, open('Results/'+name_of_result+'_probs_pickle_'+str(seed)+'.pkl',
                                'wb'))
        pickle.dump(fpr, open('Results/'+name_of_result+'_fpr_pickle_'+str(seed)+'.pkl',
                              'wb'))
        pickle.dump(tpr, open('Results/'+name_of_result+'_tpr_pickle_'+str(seed)+'.pkl',
                              'wb'))
        pickle.dump(labels, open('Results/'+name_of_result+'_labels_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        pickle.dump(auc_score, open('Results/'+name_of_result+'_auc_score_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        pickle.dump(report, open('Results/'+name_of_result+'_report_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        
print('Took ', time.time()-start, ' seconds')

../Core-scripts/prediction.py:178: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 144 
Model Report
SR 144 Accuracy: 0.7177
SR 144 AUC Score (Train): 0.740136
SR 144 Report 
              precision    recall  f1-score   support

          0       0.74      0.83      0.78        75
          1       0.68      0.55      0.61        49

avg / total       0.71      0.72      0.71       124

GEN 144 
Model Report
GEN 144 Accuracy: 0.8065
GEN 144 AUC Score (Train): 0.841905
GEN 144 Report 
              precision    recall  f1-score   support

          0       0.78      0.95      0.86        75
          1       0.88      0.59      0.71        49

avg / total       0.82      0.81      0.80       124

BE 144 
Model Report
BE 144 Accuracy: 0.75
BE 144 AUC Score (Train): 0.853061
BE 144 Report 
              precision    recall  f1-score   support

          0       0.76      0.87      0.81        75
          1       0.74      0.57      0.64        49

avg / total       0.75      0.75      0.74       124



../Core-scripts/prediction.py:178: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 235 
Model Report
SR 235 Accuracy: 0.5556
SR 235 AUC Score (Train): 0.742069
SR 235 Report 
              precision    recall  f1-score   support

          0       0.56      0.79      0.66        29
          1       0.54      0.28      0.37        25

avg / total       0.55      0.56      0.52        54

GEN 235 
Model Report
GEN 235 Accuracy: 0.7222
GEN 235 AUC Score (Train): 0.888276
GEN 235 Report 
              precision    recall  f1-score   support

          0       0.68      0.90      0.78        29
          1       0.81      0.52      0.63        25

avg / total       0.74      0.72      0.71        54

BE 235 
Model Report
BE 235 Accuracy: 0.7407
BE 235 AUC Score (Train): 0.888276
BE 235 Report 
              precision    recall  f1-score   support

          0       0.70      0.90      0.79        29
          1       0.82      0.56      0.67        25

avg / total       0.76      0.74      0.73        54



../Core-scripts/prediction.py:178: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)


...


Predicting

SR 5056 
Model Report
SR 5056 Accuracy: 0.5652
SR 5056 AUC Score (Train): 0.636684
SR 5056 Report 
              precision    recall  f1-score   support

          0       0.62      0.71      0.67        42
          1       0.43      0.33      0.38        27

avg / total       0.55      0.57      0.55        69

GEN 5056 
Model Report
GEN 5056 Accuracy: 0.6522
GEN 5056 AUC Score (Train): 0.689594
GEN 5056 Report 
              precision    recall  f1-score   support

          0       0.71      0.71      0.71        42
          1       0.56      0.56      0.56        27

avg / total       0.65      0.65      0.65        69

BE 5056 
Model Report
BE 5056 Accuracy: 0.6957
BE 5056 AUC Score (Train): 0.744268
BE 5056 Report 
              precision    recall  f1-score   support

          0       0.74      0.76      0.75        42
          1       0.62      0.59      0.60        27

avg / total       0.69      0.70      0.69        69



../Core-scripts/prediction.py:178: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 5192 
Model Report
SR 5192 Accuracy: 0.5152
SR 5192 AUC Score (Train): 0.527885
SR 5192 Report 
              precision    recall  f1-score   support

          0       0.42      0.62      0.50        26
          1       0.64      0.45      0.53        40

avg / total       0.56      0.52      0.52        66

GEN 5192 
Model Report
GEN 5192 Accuracy: 0.6061
GEN 5192 AUC Score (Train): 0.625962
GEN 5192 Report 
              precision    recall  f1-score   support

          0       0.50      0.58      0.54        26
          1       0.69      0.62      0.66        40

avg / total       0.62      0.61      0.61        66

BE 5192 
Model Report
BE 5192 Accuracy: 0.5606
BE 5192 AUC Score (Train): 0.623077
BE 5192 Report 
              precision    recall  f1-score   support

          0       0.46      0.62      0.52        26
          1       0.68      0.53      0.59        40

avg / total       0.59      0.56      0.57        66



../Core-scripts/prediction.py:178: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 7751 
Model Report
SR 7751 Accuracy: 0.6905
SR 7751 AUC Score (Train): 0.737847
SR 7751 Report 
              precision    recall  f1-score   support

          0       0.70      0.81      0.75        48
          1       0.68      0.53      0.59        36

avg / total       0.69      0.69      0.68        84

GEN 7751 
Model Report
GEN 7751 Accuracy: 0.7857
GEN 7751 AUC Score (Train): 0.754630
GEN 7751 Report 
              precision    recall  f1-score   support

          0       0.77      0.90      0.83        48
          1       0.82      0.64      0.72        36

avg / total       0.79      0.79      0.78        84

BE 7751 
Model Report
BE 7751 Accuracy: 0.75
BE 7751 AUC Score (Train): 0.816551
BE 7751 Report 
              precision    recall  f1-score   support

          0       0.72      0.92      0.81        48
          1       0.83      0.53      0.64        36

avg / total       0.77      0.75      0.74        84


Predicting

SR 7813 
Model Report
SR 78

In [10]:
import time
name_of_result = 'yeast_OLD'
strict_data = yeast_strict
w2v_strict = yeast_strict_model
w2v_gen = yeast_gen_model
w2v_be = yeast_be_model
xgb_clf = XGBClassifier(learning_rate = 0.01,
                        n_estimators = 5000,
                        seed=24)
start = time.time()
for seed in random_seeds:
        strict_list_SR = pred.make_models(strict_data,
                                          name_of_result+'_SR_'+str(seed),
                                          prev_model=w2v_strict,
                                          ran_state=seed)

        strict_list_GEN = pred.make_models(strict_data,
                                           name_of_result+'_GEN_'+str(seed),
                                           prev_model=w2v_gen,
                                           ran_state=seed)
        strict_list_BE = pred.make_models(strict_data,
                                          name_of_result+'_BE_'+str(seed),
                                          prev_model=w2v_be,
                                          ran_state=seed)

        strict_final_list = [strict_list_SR,
                             strict_list_GEN,
                             strict_list_BE]

        print ('\nPredicting\n')
        accuracy = []
        probs = []
        fpr = []
        tpr = []
        labels = []
        auc_score = []
        report = []

        for entry, model_name in zip(strict_final_list, ['SR '+str(seed), 'GEN '+str(seed), 'BE '+str(seed)]):
            accuracy_norm, auc_score_norm, pred_labels_norm, probs_norm, class_report_norm  = pred.XGB_modelfit(xgb_clf, 
                                                                                                                entry[0], 
                                                                                                                entry[2], 
                                                                                                                entry[1], 
                                                                                                                entry[3], 
                                                                                                                model_name)
            fpr_norm, tpr_norm, _ = roc_curve(entry[3], probs_norm)

            accuracy.append([accuracy_norm])
            probs.append([probs_norm])
            fpr.append([fpr_norm])
            tpr.append([tpr_norm])
            labels.append([pred_labels_norm])
            auc_score.append([auc_score_norm])
            report.append([class_report_norm])

        pickle.dump(accuracy, open('Results/'+name_of_result+'_accuracy_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        pickle.dump(probs, open('Results/'+name_of_result+'_probs_pickle_'+str(seed)+'.pkl',
                                'wb'))
        pickle.dump(fpr, open('Results/'+name_of_result+'_fpr_pickle_'+str(seed)+'.pkl',
                              'wb'))
        pickle.dump(tpr, open('Results/'+name_of_result+'_tpr_pickle_'+str(seed)+'.pkl',
                              'wb'))
        pickle.dump(labels, open('Results/'+name_of_result+'_labels_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        pickle.dump(auc_score, open('Results/'+name_of_result+'_auc_score_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        pickle.dump(report, open('Results/'+name_of_result+'_report_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        
print('Took ', time.time()-start, ' seconds')

../Core-scripts/prediction.py:178: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 144 
Model Report
SR 144 Accuracy: 0.7419
SR 144 AUC Score (Train): 0.740680
SR 144 Report 
              precision    recall  f1-score   support

          0       0.74      0.88      0.80        75
          1       0.74      0.53      0.62        49

avg / total       0.74      0.74      0.73       124

GEN 144 
Model Report
GEN 144 Accuracy: 0.8145
GEN 144 AUC Score (Train): 0.858231
GEN 144 Report 
              precision    recall  f1-score   support

          0       0.80      0.92      0.86        75
          1       0.84      0.65      0.74        49

avg / total       0.82      0.81      0.81       124

BE 144 
Model Report
BE 144 Accuracy: 0.8065
BE 144 AUC Score (Train): 0.878095
BE 144 Report 
              precision    recall  f1-score   support

          0       0.81      0.89      0.85        75
          1       0.80      0.67      0.73        49

avg / total       0.81      0.81      0.80       124



../Core-scripts/prediction.py:178: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 235 
Model Report
SR 235 Accuracy: 0.5741
SR 235 AUC Score (Train): 0.746207
SR 235 Report 
              precision    recall  f1-score   support

          0       0.58      0.76      0.66        29
          1       0.56      0.36      0.44        25

avg / total       0.57      0.57      0.56        54

GEN 235 
Model Report
GEN 235 Accuracy: 0.7222
GEN 235 AUC Score (Train): 0.884138
GEN 235 Report 
              precision    recall  f1-score   support

          0       0.68      0.90      0.78        29
          1       0.81      0.52      0.63        25

avg / total       0.74      0.72      0.71        54

BE 235 
Model Report
BE 235 Accuracy: 0.7407
BE 235 AUC Score (Train): 0.895172
BE 235 Report 
              precision    recall  f1-score   support

          0       0.70      0.90      0.79        29
          1       0.82      0.56      0.67        25

avg / total       0.76      0.74      0.73        54



../Core-scripts/prediction.py:178: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 905 
Model Report
SR 905 Accuracy: 0.5556
SR 905 AUC Score (Train): 0.619305
SR 905 Report 
              precision    recall  f1-score   support

          0       0.53      0.80      0.64        35
          1       0.63      0.32      0.43        37

avg / total       0.58      0.56      0.53        72



KeyboardInterrupt: 

In [36]:
name_of_result = 'yeast_NEW'
strict_data = yeast_strict
w2v_strict = word2vec.Word2Vec.load('../../Results/Yeast/models/yeast_strict_w2v_model')
w2v_gen = word2vec.Word2Vec.load('../../Results/Yeast/models/yeast_gen_w2v_model')
w2v_be = word2vec.Word2Vec.load('../../Results/Yeast/models/yeast_be_w2v_model')
xgb_clf = XGBClassifier(learning_rate=0.1,
                        n_estimators=1000,
                        max_depth=6,
                        min_child_weight=1,
                        gamma=0.2,
                        subsample=0.6,
                        colsample_bytree=0.8,
                        reg_alpha=0.01,
                        objective='binary:logistic',
                        scale_pos_weight=1,
                        seed=24)
start = time.time()
for seed in random_seeds:
        strict_list_SR = pred.make_models(strict_data,
                                          name_of_result+'_SR_'+str(seed),
                                          prev_model=w2v_strict,
                                          ran_state=seed)

        strict_list_GEN = pred.make_models(strict_data,
                                           name_of_result+'_GEN_'+str(seed),
                                           prev_model=w2v_gen,
                                           ran_state=seed)
        strict_list_BE = pred.make_models(strict_data,
                                          name_of_result+'_BE_'+str(seed),
                                          prev_model=w2v_be,
                                          ran_state=seed)

        strict_final_list = [strict_list_SR,
                             strict_list_GEN,
                             strict_list_BE]

        print ('\nPredicting\n')
        accuracy = []
        probs = []
        fpr = []
        tpr = []
        labels = []
        auc_score = []
        report = []

        for entry, model_name in zip(strict_final_list, ['SR '+str(seed), 'GEN '+str(seed), 'BE '+str(seed)]):
            accuracy_norm, auc_score_norm, pred_labels_norm, probs_norm, class_report_norm  = modelfit(xgb_clf, 
                                                                                                       entry[0], 
                                                                                                       entry[2], 
                                                                                                       entry[1], 
                                                                                                       entry[3], 
                                                                                                       model_name)
            fpr_norm, tpr_norm, _ = roc_curve(entry[3], probs_norm)

            accuracy.append([accuracy_norm])
            probs.append([probs_norm])
            fpr.append([fpr_norm])
            tpr.append([tpr_norm])
            labels.append([pred_labels_norm])
            auc_score.append([auc_score_norm])
            report.append([class_report_norm])

        pickle.dump(accuracy, open('Results/'+name_of_result+'_accuracy_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        pickle.dump(probs, open('Results/'+name_of_result+'_probs_pickle_'+str(seed)+'.pkl',
                                'wb'))
        pickle.dump(fpr, open('Results/'+name_of_result+'_fpr_pickle_'+str(seed)+'.pkl',
                              'wb'))
        pickle.dump(tpr, open('Results/'+name_of_result+'_tpr_pickle_'+str(seed)+'.pkl',
                              'wb'))
        pickle.dump(labels, open('Results/'+name_of_result+'_labels_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        pickle.dump(auc_score, open('Results/'+name_of_result+'_auc_score_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        pickle.dump(report, open('Results/'+name_of_result+'_report_pickle_'+str(seed)+'.pkl',
                                 'wb'))
print('Took ', time.time()-start, ' seconds')

2017-06-02 13:33:46,449 : INFO : loading Word2Vec object from ../../Results/Yeast/models/yeast_strict_w2v_model
2017-06-02 13:33:46,585 : INFO : loading wv recursively from ../../Results/Yeast/models/yeast_strict_w2v_model.wv.* with mmap=None
2017-06-02 13:33:46,585 : INFO : setting ignored attribute cum_table to None
2017-06-02 13:33:46,586 : INFO : setting ignored attribute syn0norm to None
2017-06-02 13:33:46,587 : INFO : loaded ../../Results/Yeast/models/yeast_strict_w2v_model
2017-06-02 13:33:46,592 : INFO : loading Word2Vec object from ../../Results/Yeast/models/yeast_gen_w2v_model
2017-06-02 13:33:46,937 : INFO : loading wv recursively from ../../Results/Yeast/models/yeast_gen_w2v_model.wv.* with mmap=None
2017-06-02 13:33:46,938 : INFO : setting ignored attribute syn0norm to None
2017-06-02 13:33:46,939 : INFO : setting ignored attribute cum_table to None
2017-06-02 13:33:46,939 : INFO : loaded ../../Results/Yeast/models/yeast_gen_w2v_model
2017-06-02 13:33:46,952 : INFO : load


Predicting

SR 144 
Model Report
SR 144 Accuracy: 0.6094
SR 144 AUC Score (Train): 0.637638
SR 144 Report 
              precision    recall  f1-score   support

          0       0.64      0.73      0.68        37
          1       0.55      0.44      0.49        27

avg / total       0.60      0.61      0.60        64

GEN 144 
Model Report
GEN 144 Accuracy: 0.6719
GEN 144 AUC Score (Train): 0.684685
GEN 144 Report 
              precision    recall  f1-score   support

          0       0.66      0.89      0.76        37
          1       0.71      0.37      0.49        27

avg / total       0.68      0.67      0.64        64

BE 144 
Model Report
BE 144 Accuracy: 0.6562
BE 144 AUC Score (Train): 0.727728
BE 144 Report 
              precision    recall  f1-score   support

          0       0.65      0.89      0.75        37
          1       0.69      0.33      0.45        27

avg / total       0.67      0.66      0.62        64



../Core-scripts/prediction.py:176: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 235 
Model Report
SR 235 Accuracy: 0.7101
SR 235 AUC Score (Train): 0.735993
SR 235 Report 
              precision    recall  f1-score   support

          0       0.71      0.79      0.75        38
          1       0.70      0.61      0.66        31

avg / total       0.71      0.71      0.71        69

GEN 235 
Model Report
GEN 235 Accuracy: 0.6957
GEN 235 AUC Score (Train): 0.723260
GEN 235 Report 
              precision    recall  f1-score   support

          0       0.71      0.76      0.73        38
          1       0.68      0.61      0.64        31

avg / total       0.69      0.70      0.69        69

BE 235 
Model Report
BE 235 Accuracy: 0.6812
BE 235 AUC Score (Train): 0.721562
BE 235 Report 
              precision    recall  f1-score   support

          0       0.69      0.76      0.72        38
          1       0.67      0.58      0.62        31

avg / total       0.68      0.68      0.68        69


Predicting

SR 905 
Model Report
SR 905 Accuracy:

../Core-scripts/prediction.py:176: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)


...


Predicting

SR 5192 
Model Report
SR 5192 Accuracy: 0.6582
SR 5192 AUC Score (Train): 0.759973
SR 5192 Report 
              precision    recall  f1-score   support

          0       0.55      0.84      0.67        32
          1       0.83      0.53      0.65        47

avg / total       0.72      0.66      0.66        79

GEN 5192 
Model Report
GEN 5192 Accuracy: 0.6456
GEN 5192 AUC Score (Train): 0.767287
GEN 5192 Report 
              precision    recall  f1-score   support

          0       0.54      0.78      0.64        32
          1       0.79      0.55      0.65        47

avg / total       0.69      0.65      0.65        79

BE 5192 
Model Report
BE 5192 Accuracy: 0.6709
BE 5192 AUC Score (Train): 0.754654
BE 5192 Report 
              precision    recall  f1-score   support

          0       0.56      0.88      0.68        32
          1       0.86      0.53      0.66        47

avg / total       0.74      0.67      0.67        79



../Core-scripts/prediction.py:176: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 7751 
Model Report
SR 7751 Accuracy: 0.6
SR 7751 AUC Score (Train): 0.605600
SR 7751 Report 
              precision    recall  f1-score   support

          0       0.78      0.56      0.65        50
          1       0.44      0.68      0.53        25

avg / total       0.66      0.60      0.61        75

GEN 7751 
Model Report
GEN 7751 Accuracy: 0.56
GEN 7751 AUC Score (Train): 0.694400
GEN 7751 Report 
              precision    recall  f1-score   support

          0       0.76      0.50      0.60        50
          1       0.40      0.68      0.51        25

avg / total       0.64      0.56      0.57        75

BE 7751 
Model Report
BE 7751 Accuracy: 0.6267
BE 7751 AUC Score (Train): 0.685600
BE 7751 Report 
              precision    recall  f1-score   support

          0       0.81      0.58      0.67        50
          1       0.46      0.72      0.56        25

avg / total       0.69      0.63      0.64        75



../Core-scripts/prediction.py:176: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 7813 
Model Report
SR 7813 Accuracy: 0.6234
SR 7813 AUC Score (Train): 0.688098
SR 7813 Report 
              precision    recall  f1-score   support

          0       0.56      0.65      0.60        34
          1       0.68      0.60      0.64        43

avg / total       0.63      0.62      0.62        77

GEN 7813 
Model Report
GEN 7813 Accuracy: 0.7273
GEN 7813 AUC Score (Train): 0.756498
GEN 7813 Report 
              precision    recall  f1-score   support

          0       0.67      0.76      0.71        34
          1       0.79      0.70      0.74        43

avg / total       0.74      0.73      0.73        77

BE 7813 
Model Report
BE 7813 Accuracy: 0.6883
BE 7813 AUC Score (Train): 0.720246
BE 7813 Report 
              precision    recall  f1-score   support

          0       0.67      0.59      0.62        34
          1       0.70      0.77      0.73        43

avg / total       0.69      0.69      0.69        77

Took  1404.1214997768402  seconds


In [37]:
name_of_result = 'yeast_NEW_w2v_OLD_xgb'
strict_data = yeast_strict
w2v_strict = word2vec.Word2Vec.load('../../Results/Yeast/models/yeast_strict_w2v_model')
w2v_gen = word2vec.Word2Vec.load('../../Results/Yeast/models/yeast_gen_w2v_model')
w2v_be = word2vec.Word2Vec.load('../../Results/Yeast/models/yeast_be_w2v_model')
xgb_clf = XGBClassifier(seed=24)
start = time.time()
for seed in random_seeds:
        strict_list_SR = pred.make_models(strict_data,
                                          name_of_result+'_SR_'+str(seed),
                                          prev_model=w2v_strict,
                                          ran_state=seed)

        strict_list_GEN = pred.make_models(strict_data,
                                           name_of_result+'_GEN_'+str(seed),
                                           prev_model=w2v_gen,
                                           ran_state=seed)
        strict_list_BE = pred.make_models(strict_data,
                                          name_of_result+'_BE_'+str(seed),
                                          prev_model=w2v_be,
                                          ran_state=seed)

        strict_final_list = [strict_list_SR,
                             strict_list_GEN,
                             strict_list_BE]

        print ('\nPredicting\n')
        accuracy = []
        probs = []
        fpr = []
        tpr = []
        labels = []
        auc_score = []
        report = []

        for entry, model_name in zip(strict_final_list, ['SR '+str(seed), 'GEN '+str(seed), 'BE '+str(seed)]):
            accuracy_norm, auc_score_norm, pred_labels_norm, probs_norm, class_report_norm  = modelfit(xgb_clf, 
                                                                                                       entry[0], 
                                                                                                       entry[2], 
                                                                                                       entry[1], 
                                                                                                       entry[3], 
                                                                                                       model_name)
            fpr_norm, tpr_norm, _ = roc_curve(entry[3], probs_norm)

            accuracy.append([accuracy_norm])
            probs.append([probs_norm])
            fpr.append([fpr_norm])
            tpr.append([tpr_norm])
            labels.append([pred_labels_norm])
            auc_score.append([auc_score_norm])
            report.append([class_report_norm])

        pickle.dump(accuracy, open('Results/'+name_of_result+'_accuracy_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        pickle.dump(probs, open('Results/'+name_of_result+'_probs_pickle_'+str(seed)+'.pkl',
                                'wb'))
        pickle.dump(fpr, open('Results/'+name_of_result+'_fpr_pickle_'+str(seed)+'.pkl',
                              'wb'))
        pickle.dump(tpr, open('Results/'+name_of_result+'_tpr_pickle_'+str(seed)+'.pkl',
                              'wb'))
        pickle.dump(labels, open('Results/'+name_of_result+'_labels_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        pickle.dump(auc_score, open('Results/'+name_of_result+'_auc_score_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        pickle.dump(report, open('Results/'+name_of_result+'_report_pickle_'+str(seed)+'.pkl',
                                 'wb'))
print('Took ', time.time()-start, ' seconds')

2017-06-02 13:57:11,912 : INFO : loading Word2Vec object from ../../Results/Yeast/models/yeast_strict_w2v_model
2017-06-02 13:57:12,086 : INFO : loading wv recursively from ../../Results/Yeast/models/yeast_strict_w2v_model.wv.* with mmap=None
2017-06-02 13:57:12,087 : INFO : setting ignored attribute cum_table to None
2017-06-02 13:57:12,088 : INFO : setting ignored attribute syn0norm to None
2017-06-02 13:57:12,089 : INFO : loaded ../../Results/Yeast/models/yeast_strict_w2v_model
2017-06-02 13:57:12,106 : INFO : loading Word2Vec object from ../../Results/Yeast/models/yeast_gen_w2v_model
2017-06-02 13:57:12,559 : INFO : loading wv recursively from ../../Results/Yeast/models/yeast_gen_w2v_model.wv.* with mmap=None
2017-06-02 13:57:12,561 : INFO : setting ignored attribute syn0norm to None
2017-06-02 13:57:12,561 : INFO : setting ignored attribute cum_table to None
2017-06-02 13:57:12,562 : INFO : loaded ../../Results/Yeast/models/yeast_gen_w2v_model
2017-06-02 13:57:12,582 : INFO : load


Predicting

SR 144 
Model Report
SR 144 Accuracy: 0.625
SR 144 AUC Score (Train): 0.691692
SR 144 Report 
              precision    recall  f1-score   support

          0       0.65      0.76      0.70        37
          1       0.57      0.44      0.50        27

avg / total       0.62      0.62      0.62        64

GEN 144 
Model Report
GEN 144 Accuracy: 0.7188
GEN 144 AUC Score (Train): 0.731732
GEN 144 Report 
              precision    recall  f1-score   support

          0       0.69      0.92      0.79        37
          1       0.80      0.44      0.57        27

avg / total       0.74      0.72      0.70        64

BE 144 
Model Report
BE 144 Accuracy: 0.6406
BE 144 AUC Score (Train): 0.724725
BE 144 Report 
              precision    recall  f1-score   support

          0       0.65      0.84      0.73        37
          1       0.62      0.37      0.47        27

avg / total       0.64      0.64      0.62        64



../Core-scripts/prediction.py:176: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 235 
Model Report
SR 235 Accuracy: 0.6812
SR 235 AUC Score (Train): 0.717317
SR 235 Report 
              precision    recall  f1-score   support

          0       0.70      0.74      0.72        38
          1       0.66      0.61      0.63        31

avg / total       0.68      0.68      0.68        69

GEN 235 
Model Report
GEN 235 Accuracy: 0.7246
GEN 235 AUC Score (Train): 0.719015
GEN 235 Report 
              precision    recall  f1-score   support

          0       0.72      0.82      0.77        38
          1       0.73      0.61      0.67        31

avg / total       0.73      0.72      0.72        69

BE 235 
Model Report
BE 235 Accuracy: 0.6522
BE 235 AUC Score (Train): 0.729202
BE 235 Report 
              precision    recall  f1-score   support

          0       0.68      0.71      0.69        38
          1       0.62      0.58      0.60        31

avg / total       0.65      0.65      0.65        69


Predicting

SR 905 
Model Report
SR 905 Accuracy:

../Core-scripts/prediction.py:176: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)


...


Predicting

SR 5192 
Model Report
SR 5192 Accuracy: 0.6962
SR 5192 AUC Score (Train): 0.789229
SR 5192 Report 
              precision    recall  f1-score   support

          0       0.58      0.91      0.71        32
          1       0.90      0.55      0.68        47

avg / total       0.77      0.70      0.69        79

GEN 5192 
Model Report
GEN 5192 Accuracy: 0.6835
GEN 5192 AUC Score (Train): 0.767952
GEN 5192 Report 
              precision    recall  f1-score   support

          0       0.57      0.91      0.70        32
          1       0.89      0.53      0.67        47

avg / total       0.76      0.68      0.68        79

BE 5192 
Model Report
BE 5192 Accuracy: 0.7468
BE 5192 AUC Score (Train): 0.815160
BE 5192 Report 
              precision    recall  f1-score   support

          0       0.64      0.88      0.74        32
          1       0.89      0.66      0.76        47

avg / total       0.78      0.75      0.75        79



../Core-scripts/prediction.py:176: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 7751 
Model Report
SR 7751 Accuracy: 0.64
SR 7751 AUC Score (Train): 0.672800
SR 7751 Report 
              precision    recall  f1-score   support

          0       0.79      0.62      0.70        50
          1       0.47      0.68      0.56        25

avg / total       0.69      0.64      0.65        75

GEN 7751 
Model Report
GEN 7751 Accuracy: 0.6
GEN 7751 AUC Score (Train): 0.663200
GEN 7751 Report 
              precision    recall  f1-score   support

          0       0.76      0.58      0.66        50
          1       0.43      0.64      0.52        25

avg / total       0.65      0.60      0.61        75

BE 7751 
Model Report
BE 7751 Accuracy: 0.6267
BE 7751 AUC Score (Train): 0.670400
BE 7751 Report 
              precision    recall  f1-score   support

          0       0.78      0.62      0.69        50
          1       0.46      0.64      0.53        25

avg / total       0.67      0.63      0.64        75



../Core-scripts/prediction.py:176: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

SR 7813 
Model Report
SR 7813 Accuracy: 0.6623
SR 7813 AUC Score (Train): 0.704514
SR 7813 Report 
              precision    recall  f1-score   support

          0       0.61      0.65      0.63        34
          1       0.71      0.67      0.69        43

avg / total       0.66      0.66      0.66        77

GEN 7813 
Model Report
GEN 7813 Accuracy: 0.6623
GEN 7813 AUC Score (Train): 0.760602
GEN 7813 Report 
              precision    recall  f1-score   support

          0       0.61      0.65      0.63        34
          1       0.71      0.67      0.69        43

avg / total       0.66      0.66      0.66        77

BE 7813 
Model Report
BE 7813 Accuracy: 0.6494
BE 7813 AUC Score (Train): 0.751026
BE 7813 Report 
              precision    recall  f1-score   support

          0       0.60      0.62      0.61        34
          1       0.69      0.67      0.68        43

avg / total       0.65      0.65      0.65        77

Took  1503.6061532497406  seconds


In [12]:
name_of_result = 'yeast_OLD_w2v_NEW_XGB'
strict_data = yeast_strict
w2v_strict = yeast_strict_model
w2v_gen = yeast_gen_model
w2v_be = yeast_be_model
xgb_clf = XGBClassifier(learning_rate=0.1,
                        n_estimators=1000,
                        max_depth=6,
                        min_child_weight=1,
                        gamma=0.2,
                        subsample=0.6,
                        colsample_bytree=0.8,
                        reg_alpha=0.01,
                        objective='binary:logistic',
                        scale_pos_weight=1,
                        seed=24)
start = time.time()
for seed in random_seeds:
        strict_list_SR = pred.make_models(strict_data,
                                          name_of_result+'_SR_'+str(seed),
                                          prev_model=w2v_strict,
                                          ran_state=seed)

        strict_list_GEN = pred.make_models(strict_data,
                                           name_of_result+'_GEN_'+str(seed),
                                           prev_model=w2v_gen,
                                           ran_state=seed)
        strict_list_BE = pred.make_models(strict_data,
                                          name_of_result+'_BE_'+str(seed),
                                          prev_model=w2v_be,
                                          ran_state=seed)

        strict_final_list = [strict_list_SR,
                             strict_list_GEN,
                             strict_list_BE]

        print ('\nPredicting\n')
        accuracy = []
        probs = []
        fpr = []
        tpr = []
        labels = []
        auc_score = []
        report = []

        for entry, model_name in zip(strict_final_list, ['SR '+str(seed), 'GEN '+str(seed), 'BE '+str(seed)]):
            accuracy_norm, auc_score_norm, pred_labels_norm, probs_norm, class_report_norm  = modelfit(xgb_clf, 
                                                                                                       entry[0], 
                                                                                                       entry[2], 
                                                                                                       entry[1], 
                                                                                                       entry[3], 
                                                                                                       model_name)
            fpr_norm, tpr_norm, _ = roc_curve(entry[3], probs_norm)

            accuracy.append([accuracy_norm])
            probs.append([probs_norm])
            fpr.append([fpr_norm])
            tpr.append([tpr_norm])
            labels.append([pred_labels_norm])
            auc_score.append([auc_score_norm])
            report.append([class_report_norm])

        pickle.dump(accuracy, open('Results/'+name_of_result+'_accuracy_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        pickle.dump(probs, open('Results/'+name_of_result+'_probs_pickle_'+str(seed)+'.pkl',
                                'wb'))
        pickle.dump(fpr, open('Results/'+name_of_result+'_fpr_pickle_'+str(seed)+'.pkl',
                              'wb'))
        pickle.dump(tpr, open('Results/'+name_of_result+'_tpr_pickle_'+str(seed)+'.pkl',
                              'wb'))
        pickle.dump(labels, open('Results/'+name_of_result+'_labels_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        pickle.dump(auc_score, open('Results/'+name_of_result+'_auc_score_pickle_'+str(seed)+'.pkl',
                                 'wb'))
        pickle.dump(report, open('Results/'+name_of_result+'_report_pickle_'+str(seed)+'.pkl',
                                 'wb'))
print('Took ', time.time()-start, ' seconds')


Predicting

SR 144 
Model Report
SR 144 Accuracy: 0.4744
SR 144 AUC Score (Train): 0.537542
SR 144 Report 
              precision    recall  f1-score   support

          0       0.45      0.71      0.55        35
          1       0.55      0.28      0.37        43

avg / total       0.50      0.47      0.45        78

GEN 144 
Model Report
GEN 144 Accuracy: 0.6154
GEN 144 AUC Score (Train): 0.657143
GEN 144 Report 
              precision    recall  f1-score   support

          0       0.54      0.89      0.67        35
          1       0.81      0.40      0.53        43

avg / total       0.69      0.62      0.60        78

BE 144 
Model Report
BE 144 Accuracy: 0.6667
BE 144 AUC Score (Train): 0.714286
BE 144 Report 
              precision    recall  f1-score   support

          0       0.59      0.86      0.70        35
          1       0.81      0.51      0.63        43

avg / total       0.71      0.67      0.66        78


Predicting



KeyboardInterrupt: 